# **1. Imports**

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.models as models
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
!pip install segmentation-models-pytorch --quiet
import segmentation_models_pytorch as smp
from tqdm import tqdm
from sklearn.model_selection import KFold
import albumentations as A
from albumentations import (Compose, ShiftScaleRotate, Resize, RandomRotate90,
                            VerticalFlip, HorizontalFlip, OneOf, ElasticTransform,
                            GridDistortion, OpticalDistortion, CLAHE,
                            GaussNoise, ISONoise, RandomBrightnessContrast, RandomGamma)
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
import time
!pip install torch-lr-finder
from torch_lr_finder import LRFinder
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 7.2 MB/s eta 0:00:00


# **2. Data Prep**

**2.1 Load and Process the DataFrame**

In [2]:
# Load the train.csv file
train_df = pd.read_csv('/kaggle/input/understanding_cloud_organization/train.csv')

# Separate Image_Label into ImageId and Label
train_df[['ImageId', 'Label']] = train_df['Image_Label'].str.split('_', expand=True)
train_df = train_df.drop(columns=['Image_Label'])

# Fill missing EncodedPixels with NaN
train_df['EncodedPixels'] = train_df['EncodedPixels'].fillna('')

# Pivot the dataframe to have one row per image
train_df = train_df.pivot(index='ImageId', columns='Label', values='EncodedPixels').reset_index()

# Fill missing values with empty strings
train_df.fillna('', inplace=True)

print(train_df.head())


Label      ImageId                                               Fish  \
0      0011165.jpg  264918 937 266318 937 267718 937 269118 937 27...   
1      002be4f.jpg  233813 878 235213 878 236613 878 238010 881 23...   
2      0031ae9.jpg  3510 690 4910 690 6310 690 7710 690 9110 690 1...   
3      0035239.jpg                                                      
4      003994e.jpg  2367966 18 2367985 2 2367993 8 2368002 62 2369...   

Label                                             Flower  \
0      1355565 1002 1356965 1002 1358365 1002 1359765...   
1      1339279 519 1340679 519 1342079 519 1343479 51...   
2      2047 703 3447 703 4847 703 6247 703 7647 703 9...   
3      100812 462 102212 462 103612 462 105012 462 10...   
4                                                          

Label                                             Gravel  \
0                                                          
1                                                          
2                   

**2.2 Split Data into Training and Validation Sets**

In [3]:
# Initialize the 'fold' column to -1
train_df['fold'] = -1

# Initialize KFold
kf = KFold(n_splits=9, shuffle=True, random_state=42) ##maybe change THIS                              IMPORTANT

# Assign fold numbers
for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
    train_df.loc[val_idx, 'fold'] = fold

# Now select one fold for validation (e.g., fold 0)
train_data = train_df[train_df['fold'] != 0].reset_index(drop=True)
val_data = train_df[train_df['fold'] == 0].reset_index(drop=True)

# Verify the split
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")


Training data size: 4929
Validation data size: 617


# **3. Create Custom Dataset**

In [4]:
class CloudDataset(Dataset):
    def __init__(self, df, img_dir, mask_dir=None, transform=None, resize_shape=(416, 608), mode='train'):
        self.df = df
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.resize_shape = resize_shape
        self.mode = mode  # 'train', 'val', or 'test'
        self.labels = ['Fish', 'Flower', 'Gravel', 'Sugar']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get image ID
        img_id = self.df.iloc[idx]['ImageId']
        # Load image
        img_path = os.path.join(self.img_dir, img_id)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.mode != 'test':
            # Create mask
            mask = np.zeros((1400, 2100, 4), dtype=np.float32)
            for i, label in enumerate(self.labels):
                rle = self.df.iloc[idx][label]
                if rle != '':
                    mask[:, :, i] = rle2mask(rle, (1400, 2100))
            # Resize image and mask
            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']
            else:
                # Resize
                image = cv2.resize(image, (self.resize_shape[1], self.resize_shape[0]))
                mask = cv2.resize(mask, (self.resize_shape[1], self.resize_shape[0]))
            # Transpose to channel-first
            image = image.transpose(2, 0, 1).astype(np.float32) / 255.0
            mask = mask.transpose(2, 0, 1).astype(np.float32)
            return torch.tensor(image), torch.tensor(mask)
        else:
            # For test mode
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            else:
                image = cv2.resize(image, (self.resize_shape[1], self.resize_shape[0]))
            image = image.transpose(2, 0, 1).astype(np.float32) / 255.0
            return torch.tensor(image), img_id

def rle2mask(rle, shape):
    '''
    Convert RLE(run length encoding) string to numpy array

    Parameters:
    rle (str): Run length encoding string
    shape (tuple): (height, width) of array to return

    Returns:
    numpy.array: Mask array
    '''
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1  # Convert to zero-based indexing
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')  # Reshape to the original shape


# **4. Augmentations**

In [5]:
train_transform = Compose([
    ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.6, border_mode=0),
    OneOf([
        GridDistortion(p=0.5),
        OpticalDistortion(p=0.5, distort_limit=0.4, shift_limit=0.5)
    ], p=0.8),
    RandomRotate90(p=0.5),
    VerticalFlip(p=0.5),
    HorizontalFlip(p=0.5),
    OneOf([
         CLAHE(p=0.8),
        GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        # GaussianBlur(blur_limit=3, p=0.5),
        ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.3),
    ], p=0.8),
    RandomBrightnessContrast(p=0.8),
    RandomGamma(p=0.8),
    Resize(416, 608)
])

val_transform = Compose([
    Resize(416, 608),
])


# **5. Dataloaders**

In [6]:
# Paths to your images and masks
train_img_dir = '/kaggle/input/understanding_cloud_organization/train_images'
test_img_dir = '/kaggle/input/understanding_cloud_organization/test_images'

# Create datasets
train_dataset = CloudDataset(df=train_data, img_dir=train_img_dir, transform=train_transform)
val_dataset = CloudDataset(df=val_data, img_dir=train_img_dir, transform=val_transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, pin_memory=True)

print(len(train_loader))
print(len(val_loader))

2465
309


# **6. Model**

In [7]:
import os
import sys
import torch
import subprocess

# Step 1: Ensure U-2-Net is Cloned
if not os.path.exists("U-2-Net"):
    print("⏳ Cloning U²-Net repository...")
    subprocess.run(["git", "clone", "https://github.com/xuebinqin/U-2-Net.git"])
    print("✅ U²-Net repository cloned!")
else:
    print("✅ U-2-Net is already cloned. Skipping download.")

# Step 2: Add the correct path for model loading
sys.path.append(os.path.abspath("U-2-Net/model"))  # Add 'model' folder to Python path

# Step 3: Import the U²-Net Model Correctly
from u2net import U2NET  # Directly import the U2NET class

# Step 4: Initialize U²-Net Model
model = U2NET()

# Step 5: Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 6: Load Pretrained Weights from Correct Path
weights_path = "/kaggle/working/saved_models/u2net/u2net.pth"

if os.path.exists(weights_path):
    print("⏳ Loading pretrained U²-Net weights...")
    model.load_state_dict(torch.load(weights_path, map_location=device))
    print("✅ U²-Net model loaded successfully with pretrained weights!")
else:
    print("❌ ERROR: Weights file not found! Check the path.")

⏳ Cloning U²-Net repository...
✅ U²-Net repository cloned!
❌ ERROR: Weights file not found! Check the path.


# **7. Loss Function and Metrics**

In [8]:
# Define Dice Loss
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)  # Apply sigmoid to get probabilities
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        return 1 - dice

#Define BCE + Dice Loss
class BCEDiceLoss(nn.Module):
    def __init__(self):
        super(BCEDiceLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        dice_loss = self.dice(inputs, targets)
        return bce_loss + dice_loss

# **8. Training**

**8.1 Early Stop Definition**

In [9]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        """
        Early stops the training if validation loss doesn't improve after a given patience.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta  # Minimum change to qualify as an improvement

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


**8.2 Hyperparameters**

In [10]:
# Optimizer
criterion = BCEDiceLoss()
optimizer = torch.optim.AdamW(
    model.parameters(),  # Ensure 'parameters()' is properly defined
    lr=6e-4,
    weight_decay=1e-4
)

# ReduceLROnPlateau Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=4, verbose=True, min_lr=1e-6
)

# Training Parameters
num_epochs = 50
best_loss = np.inf
model = model.to('cuda')

# Initialize Early Stopping
early_stopping = EarlyStopping(patience=5, verbose=True)

In [11]:
# Move model to GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

**8.3 Training Loop**

In [13]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for images, masks in train_loader:
            images = images.to('cuda')
            masks = masks.to('cuda')
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})
            pbar.update(1)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to('cuda')
            masks = masks.to('cuda')
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    # Scheduler step
    scheduler.step(avg_val_loss)

    # Evaluate 
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    # Save the model if validation loss has decreased
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print('Validation loss decreased. Model saved!')

    # Check early stopping
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print('Early stopping triggered. Stopping training.')
        break

Epoch 1/50:   0%|          | 0/2465 [00:00<?, ?batch/s]


ValueError: Target size (torch.Size([2, 416, 608])) must be the same as input size (torch.Size([2, 1, 416, 608]))

In [ ]:
# Define the path to your saved model state dictionary
#model_path = '/kaggle/input/best_model1/pytorch/default/1/best_model.pth'

# Initialize the model with the same architecture and settings
#model = smp.Unet(
#    encoder_name="efficientnet-b1",        # Same encoder (ResNet34)
#    encoder_weights="imagenet",     # Same pre-trained weights
#    in_channels=3,                  # Input channels (RGB)
#    classes=4                   
#)

# Load the model state dictionary safely with `weights_only=True`
#state_dict = torch.load(model_path, map_location='cpu', weights_only=True)
#model.load_state_dict(state_dict)

# Move model to GPU if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)

# **9. Thresholds**

**Understanding Label and Pixel Thresholds**
Before we proceed, let's clarify what we mean by label thresholds and pixel thresholds in this context:

**Label Thresholds:** These thresholds are used to decide whether a particular class (cloud type) is present in the image at all. If the maximum probability for a class is below its label threshold, we consider that the class is not present, and we skip generating a mask for it.

**Pixel Thresholds:** These thresholds are used to binarize the predicted probability maps for each class into binary masks. Pixels with probabilities above the pixel threshold are considered part of the mask; otherwise, they're considered background.

In [ ]:
# Define thresholds
initial_label_thresholds = [0.8, 0.8, 0.8, 0.8]
initial_pixel_thresholds = [0.2, 0.4, 0.4, 0.3]

efe_label_thresholds = [0.85, 0.92, 0.85, 0.85]
efe_pixel_thresholds = [0.21, 0.44, 0.4, 0.4]

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
model = model.to('cuda')

# **10. Define Metrics & Evaluate**

**10.1 Define Metrics**

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dice_coef_metric(y_pred, y_true, smooth=1):
    '''
    Calculate Dice Coefficient.

    Parameters:
    y_pred (numpy.array): Predicted mask.
    y_true (numpy.array): True mask.

    Returns:
    float: Dice coefficient.
    '''
    y_pred_f = y_pred.flatten()
    y_true_f = y_true.flatten()
    intersection = np.sum(y_pred_f * y_true_f)
    dice = (2. * intersection + smooth) / (np.sum(y_pred_f) + np.sum(y_true_f) + smooth)
    return dice

def iou_metric(y_pred, y_true, smooth=1):
    '''
    Calculate Intersection over Union (IoU).

    Parameters:
    y_pred (numpy.array): Predicted mask.
    y_true (numpy.array): True mask.

    Returns:
    float: IoU score.
    '''
    y_pred_f = y_pred.flatten()
    y_true_f = y_true.flatten()
    intersection = np.sum(y_pred_f * y_true_f)
    union = np.sum(y_pred_f) + np.sum(y_true_f) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou

**10.2 Evaluate Model**

In [ ]:
def evaluate(model, loader, label_thresholds, pixel_thresholds):
    model.eval()
    num_classes = 4
    dice_scores = []
    iou_scores = []
    
    with torch.no_grad():
        for images, masks in loader:
            images = images.to('cuda')
            masks = masks.to('cuda')
            outputs = model(images)
            outputs = torch.sigmoid(outputs)
            preds = outputs.cpu().numpy()
            trues = masks.cpu().numpy()
            batch_size = preds.shape[0]
            
            # Apply thresholds and calculate metrics
            for i in range(batch_size):
                pred = preds[i]
                true = trues[i]
                dice = []
                iou = []
                for ch in range(num_classes):
                    pred_mask = pred[ch]
                    true_mask = true[ch]
                    
                    # Apply label threshold
                    max_prob = pred_mask.max()
                    if max_prob < label_thresholds[ch]:
                        # If max probability is below label threshold, consider class absent
                        pred_mask = np.zeros_like(pred_mask)
                    else:
                        # Apply pixel threshold
                        pred_mask = (pred_mask > pixel_thresholds[ch]).astype(np.uint8)
                    
                    # Calculate Dice coefficient
                    intersection = np.logical_and(pred_mask, true_mask).sum()
                    total = pred_mask.sum() + true_mask.sum()
                    dice_score = (2 * intersection + 1e-7) / (total + 1e-7)
                    dice.append(dice_score)
                    
                    # Calculate IoU
                    union = pred_mask.sum() + true_mask.sum() - intersection
                    iou_score = (intersection + 1e-7) / (union + 1e-7)
                    iou.append(iou_score)
                    
                dice_scores.append(dice)
                iou_scores.append(iou)
                
    # Convert lists to numpy arrays
    dice_scores = np.array(dice_scores)
    iou_scores = np.array(iou_scores)
    
    # Calculate mean scores per class
    mean_dice_per_class = np.mean(dice_scores, axis=0)
    mean_iou_per_class = np.mean(iou_scores, axis=0)
    
    # Calculate overall mean scores
    mean_dice = np.mean(mean_dice_per_class)
    mean_iou = np.mean(mean_iou_per_class)
    
    return mean_dice_per_class, mean_dice, mean_iou_per_class, mean_iou


In [ ]:
# Evaluate the model before threshold optimization
print("Evaluating model before threshold optimization on full validation set...")
mean_dice_per_class, mean_dice, mean_iou_per_class, mean_iou = evaluate(
    model, val_loader, initial_label_thresholds, initial_pixel_thresholds
)

# Define class labels
labels = ['Fish', 'Flower', 'Gravel', 'Sugar']

# Print per-class Dice and IoU
for idx, label in enumerate(labels):
    print(f'{label} - Dice: {mean_dice_per_class[idx]:.4f}, IoU: {mean_iou_per_class[idx]:.4f}')

# Print overall mean Dice and IoU
print(f'Overall Mean Dice: {mean_dice:.4f}')
print(f'Overall Mean IoU: {mean_iou:.4f}')

# **11. Threshold Optimization**

**note: this takes too long :D**

In [ ]:
# def optimize_thresholds(model, loader, initial_label_thresholds, initial_pixel_thresholds):
#     best_label_thresholds = initial_label_thresholds.copy()
#     best_pixel_thresholds = initial_pixel_thresholds.copy()
#     best_dice_scores = np.zeros(4)
#     best_iou_scores = np.zeros(4)
    
#     # Define threshold ranges
#     label_threshold_ranges = [np.linspace(0.84, 0.93, 10) for _ in range(4)]
#     pixel_threshold_ranges = [np.linspace(0.2, 0.5, 10) for _ in range(4)]
    
#     # Optimize label thresholds
#     print("Optimizing label thresholds...")
#     for idx in range(4):
#         best_dice_score = 0
#         best_iou_score = 0
#         for t in label_threshold_ranges[idx]:
#             temp_label_thresholds = best_label_thresholds.copy()
#             temp_label_thresholds[idx] = t
#             mean_dice_per_class, mean_dice, mean_iou_per_class, mean_iou = evaluate(
#                 model, loader, temp_label_thresholds, best_pixel_thresholds
#             )
#             # Update if better Dice score is achieved
#             if mean_dice > best_dice_score:
#                 best_dice_score = mean_dice
#                 best_iou_score = mean_iou
#                 best_label_thresholds[idx] = t
#                 best_dice_scores[idx] = mean_dice
#                 best_iou_scores[idx] = mean_iou
#         print(f'Best label threshold for class {idx}: {best_label_thresholds[idx]}, '
#               f'Dice: {best_dice_scores[idx]:.4f}, IoU: {best_iou_scores[idx]:.4f}')
    
#     # Optimize pixel thresholds
#     print("Optimizing pixel thresholds...")
#     for idx in range(4):
#         best_dice_score = 0
#         best_iou_score = 0
#         for t in pixel_threshold_ranges[idx]:
#             temp_pixel_thresholds = best_pixel_thresholds.copy()
#             temp_pixel_thresholds[idx] = t
#             mean_dice_per_class, mean_dice, mean_iou_per_class, mean_iou = evaluate(
#                 model, loader, best_label_thresholds, temp_pixel_thresholds
#             )
#             # Update if better Dice score is achieved
#             if mean_dice > best_dice_score:
#                 best_dice_score = mean_dice
#                 best_iou_score = mean_iou
#                 best_pixel_thresholds[idx] = t
#                 best_dice_scores[idx] = mean_dice
#                 best_iou_scores[idx] = mean_iou
#         print(f'Best pixel threshold for class {idx}: {best_pixel_thresholds[idx]}, '
#               f'Dice: {best_dice_scores[idx]:.4f}, IoU: {best_iou_scores[idx]:.4f}')
    
#     return best_label_thresholds, best_pixel_thresholds

# # Optimize thresholds on the full validation set
# best_label_thresholds, best_pixel_thresholds = optimize_thresholds(
#     model, val_loader, initial_label_thresholds, initial_pixel_thresholds
# )

# print('Optimized label thresholds:', best_label_thresholds)
# print('Optimized pixel thresholds:', best_pixel_thresholds)


# **12. Submission**

In [ ]:
def mask2rle(mask):
    '''
    Convert mask to RLE.
    mask: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = mask.flatten(order='F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle

def create_submission(model, label_thresholds, pixel_thresholds, submission_file='submission.csv'):
    test_images = os.listdir(test_img_dir)
    results = []
    model.eval()
    with torch.no_grad():
        for img_name in tqdm(test_images):
            # Load image
            img_path = os.path.join(test_img_dir, img_name)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Resize
            image = cv2.resize(image, (608, 416))
            image = image.transpose(2, 0, 1).astype(np.float32) / 255.0
            image = torch.tensor(image).unsqueeze(0).to('cuda')
            # Predict
            output = model(image)
            output = torch.sigmoid(output).cpu().numpy()[0]
            # Initialize empty list for masks
            masks = []
            labels = ['Fish', 'Flower', 'Gravel', 'Sugar']
            for ch in range(4):
                pred_mask = output[ch]
                # Apply label threshold
                max_prob = pred_mask.max()
                if max_prob < label_thresholds[ch]:
                    # If max probability is below label threshold, consider class absent
                    pred_mask = np.zeros_like(pred_mask)
                else:
                    # Apply pixel threshold
                    pred_mask = (pred_mask > pixel_thresholds[ch]).astype(np.uint8)
                # Resize to 350x525
                pred_mask = cv2.resize(pred_mask, (525, 350))
                masks.append(pred_mask)
            # Convert masks to RLE
            for i, label in enumerate(labels):
                pred_mask = masks[i]
                if pred_mask.sum() == 0:
                    rle = ''
                else:
                    rle = mask2rle(pred_mask)
                results.append({'Image_Label': f'{img_name}_{label}', 'EncodedPixels': rle})
    submission_df = pd.DataFrame(results)
    # Fill missing EncodedPixels with empty strings
    submission_df['EncodedPixels'] = submission_df['EncodedPixels'].fillna('')
    submission_df.to_csv(submission_file, index=False)
    print('Submission file created!')


In [ ]:
# Create submission
create_submission(model, initial_label_thresholds, initial_pixel_thresholds, submission_file='submission1.csv')

In [ ]:
# # Create submission
# create_submission(model, best_label_thresholds, best_pixel_thresholds, submission_file='submission2.csv')

In [ ]:
# create efe
create_submission(model, efe_label_thresholds, efe_pixel_thresholds, submission_file='submission3.csv')